In [ ]:
# Import libraries
import math

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(123)

import scipy as sp

import pandas as pd

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
raster = 200
crime_type = "E05"
date_from = "2013-06-16"
date_to = "2017-03-08"
dist = 5

X = ['id', 'x200', 'y200', 'idx200', 'idy200', 'Y', 'Mo', 'W','WD', 'D', 'MoCat', 
     'vzdbcs', 'vzdpostabanka', 'lokalitarelevance','vzdprodejna','vzdrestaurace', 
     'datodHCat_1', 'datodHCat_2', 'datodHCat_3', 'datodHCat_4','datodHCat_5', 
     'stavobj_A', 'stavobj_B', 'stavobj_C', 'stavobj_D', 'stavobj_F', 'stavobj_G',
      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
    '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
    '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
    '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
    '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57',
    '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68',
    '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79',
    '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90',
    '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101',
    '102', '103', '104', '105', '106', '107', '108', '109', '110',
    '111', '112', '113', '114', '115', '116', '117', '118', '119',
    '120', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 
    'd11', 'd12', 'd13', 'd14','d15', 'd16', 'd17', 'd18', 'd19', 'd20', 'd21', 
     'PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4']

y = 'crimecount_x'

# insert main dir
main_dir = ""

# Data preprocess+timeseries+neighbours
data_dir = "%s/output/python/complet_tm_neigh_%d_%s_%s_%s_x_%d_s_select_pca.csv" % (main_dir, raster,crime_type, date_from, date_to, dist)
data = pd.read_csv(data_dir)
print(data.shape)
data.head()

In [ ]:
# http://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/
from sklearn.datasets import load_boston
from sklearn.linear_model import (LinearRegression, Ridge, 
                                  Lasso, RandomizedLasso)
from sklearn.feature_selection import RFE, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from minepy import MINE
 
np.random.seed(0)
 
names = X
 
ranks = {}
 
def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 6), ranks)
    return dict(zip(names, ranks ))
 
lr = LinearRegression(normalize=True)
lr.fit(data[X], data[y])
ranks["Linear reg"] = rank_to_dict(np.abs(lr.coef_), names)

print("Linear regression done.")
 
ridge = Ridge(alpha=7)
ridge.fit(data[X], data[y])
ranks["Ridge"] = rank_to_dict(np.abs(ridge.coef_), names)

print("Ridge regression done.")
 
lasso = Lasso(alpha=.05)
lasso.fit(data[X], data[y])
ranks["Lasso"] = rank_to_dict(np.abs(lasso.coef_), names)

print("Lasso done.")
 
rlasso = RandomizedLasso(alpha=0.04)
rlasso.fit(data[X], data[y])
ranks["Stability"] = rank_to_dict(np.abs(rlasso.scores_), names)

print("Stability done.")
 
#stop the search when 5 features are left (they will get equal scores)
rfe = RFE(lr, n_features_to_select=5)
rfe.fit(data[X],data[y])
ranks["RFE"] = rank_to_dict([float(x) for x in rfe.ranking_], names, order=-1)

print("RFE done.")

rf = RandomForestRegressor()
rf.fit(data[X],data[y])
ranks["RF"] = rank_to_dict(rf.feature_importances_, names)

print("RF done.")
 
f, pval  = f_regression(data[X], data[y], center=True)
ranks["Corr."] = rank_to_dict(f, names)

print("Correlation done.")
 
r = {}
for name in names:
    r[name] = round(np.mean([ranks[method][name] 
                             for method in ranks.keys()]), 2)
    
print("Mean done")

methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")
 
print("Done.")

In [ ]:
file_dir  = "%s/output/python/feature_selection.csv" % (main_dir)
print("\t%s" % "\t".join(methods))

with open(file_dir, "w") as file:
    for name in names:
        file.write("%s,%s\n" % (name, ",".join(map(str, [ranks[method][name] for method in methods]))))
    